In [35]:
from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobServiceClient
from io import BytesIO
import os
import sys
import yaml
import pandas as pd

#root_path = os.path.abspath(os.path.join(os.getcwd(), '.'))
#sys.path.append(root_path)

#from utils.data_collection import new_data_check, preprocess_df, make_train_test

# Don't forget to run ``az login`` in the command prompt and authenticate!

with open("../config.yml", "r") as f:
    config = yaml.safe_load(f)

ACCOUNT_URL = config['data']['account_url']
CONTAINER_NAME = config['data']['azure_container_name']
ONLINE_ADDRESS = config['data']['online_address']
DATA_PATH = config['data']['current_table']
OLD_DATA_PATH = config['data']['path_old_data']
default_credential = DefaultAzureCredential()

# Create the BlobServiceClient object
blob_service_client = BlobServiceClient(ACCOUNT_URL,
                                        credential=default_credential)
container_client = blob_service_client.get_container_client(CONTAINER_NAME)

In [36]:
import os
os.getcwd()

'c:\\Users\\marcv\\OneDrive\\MLOps\\mlops_fire_fighter\\experiments'

In [37]:
blob_list = container_client.list_blobs()
files = []
for blob in blob_list:
    files.append(blob.name)

In [34]:
len(files)

3

In [52]:
import re

results_oldies_search = []
results_data_path_search = []
oldies_paths = re.compile('oldies/')
data_path = re.compile(f'{DATA_PATH}')
older_dataframes_paths = []
for file in files:
    result_oldies = oldies_paths.search(file)
    if result_oldies is not None:
        older_dataframes_paths.append(file)
    result_data_path = data_path.search(file)
    results_oldies_search.append(result_oldies)
    results_data_path_search.append(result_data_path)

In [53]:
len([x for x in results_data_path_search if x is not None]) == 1

True

In [54]:
older_dataframes_paths

['oldies/table_2020-05-01-2023-04-30.parquet']

In [57]:
older_dataframes = []
for old_pd in older_dataframes_paths:
    df = pd.read_parquet(BytesIO(container_client.download_blob(old_pd).readall()))
    older_dataframes.append(df)

current_table = pd.read_parquet(BytesIO(container_client.download_blob(DATA_PATH).readall()))


In [58]:
for df in older_dataframes:
    assert set(df['IncidentNumber'])\
        .intersection(set(current_table['IncidentNumber'])) == set(
        df['IncidentNumber'])

    assert set(df.columns) == set(current_table.columns)
    assert len(df['IncidentNumber'].unique()) < len(
        current_table['IncidentNumber'].unique()

In [71]:
current_table

,IncidentNumber,DateOfCall,CalYear,TimeOfCall,HourOfCall,IncidentGroup,StopCodeDescription,SpecialServiceType,PropertyCategory,PropertyType,...,FirstPumpArriving_AttendanceTime,FirstPumpArriving_DeployedFromStation,SecondPumpArriving_AttendanceTime,SecondPumpArriving_DeployedFromStation,NumStationsWithPumpsAttending,NumPumpsAttending,PumpCount,PumpHoursRoundUp,Notional Cost (£),NumCalls
0,000001-01012020,01 Jan 2020,2020,00:02:11,0,Special Service,Special Service,Lift Release,Dwelling,Purpose Built Flats/Maisonettes - 4 to 9 storeys,...,NaN,None,NaN,None,1.0,1.0,1.0,1.0,339.0,3.0
1,000003-01012020,01 Jan 2020,2020,00:06:30,0,Fire,Primary Fire,None,Non Residential,Private Garden Shed,...,324.0,Hillingdon,292.0,Hillingdon,1.0,2.0,2.0,1.0,339.0,3.0
2,000004-01012020,01 Jan 2020,2020,00:07:07,0,Special Service,Special Service,Lift Release,Dwelling,Purpose Built Flats/Maisonettes - 4 to 9 storeys,...,NaN,None,NaN,None,1.0,1.0,1.0,1.0,339.0,1.0
3,000008-01012020,01 Jan 2020,2020,00:11:05,0,False Alarm,AFA,None,Non Residential,Telephone exchange,...,195.0,Richmond,NaN,None,1.0,1.0,1.0,1.0,339.0,1.0
4,000009-01012020,01 Jan 2020,2020,00:11:16,0,Fire,Secondary Fire,None,Outdoor Structure,Small refuse/rubbish container,...,426.0,Edmonton,NaN,None,1.0,1.0,1.0,1.0,339.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381361,077345-31052023,31 May 2023,2023,23:21:16,23,False Alarm,AFA,None,Dwelling,Converted Flat/Maisonettes - 3 or more storeys,...,365.0,Hammersmith,387.0,Hammersmith,1.0,2.0,2.0,1.0,388.0,1.0
381362,077346-31052023,31 May 2023,2023,23:24:02,23,Special Service,Special Service,RTC,Road Vehicle,Van,...,236.0,Addington,NaN,None,1.0,1.0,1.0,1.0,388.0,1.0
381363,077350-31052023,31 May 2023,2023,23:40:07,23,False Alarm,AFA,None,Dwelling,Converted Flat/Maisonette - Up to 2 storeys,...,337.0,Acton,388.0,Chiswick,2.0,2.0,2.0,1.0,388.0,1.0
381364,077351-31052023,31 May 2023,2023,23:41:46,23,Special Service,Special Service,RTC,Road Vehicle,Motorcycle,...,328.0,Feltham,NaN,None,1.0,1.0,1.0,1.0,388.0,1.0


In [66]:
from pathlib import Path

train_path = Path("../data/train.pkl")
test_path = Path("../data/test.pkl")

assert train_path.is_file()
assert test_path.is_file()

train = pd.read_pickle(train_path)
test = pd.read_pickle(test_path)
assert ((pd.to_datetime(current_table['DateOfCall']).max() -
            test['DateOfCall']).dt.days).all() <= 365
assert ((pd.to_datetime(train['DateOfCall']).max() -
            train['DateOfCall']).dt.days).all() >= 366
assert (train['DateOfCall'].max() - train['DateOfCall']).dt.days <= 1095

AssertionError: 

In [62]:
os.getcwd()

'c:\\Users\\marcv\\OneDrive\\MLOps\\mlops_fire_fighter\\experiments'

In [67]:
train["DateOfCall"].max()

Timestamp('2022-05-30 00:00:00')

In [70]:
train["DateOfCall"].min()

Timestamp('2020-05-31 00:00:00')